# Scoring Matrices

In [1]:
from Bio import AlignIO

# read in alignments
alignments = AlignIO.read('msa.phy', 'phylip')
print(alignments)

SingleLetterAlphabet() alignment with 5 rows and 13 columns
AACGTGGCCACAT Alpha
AAGGTCGCCACAC Beta
CAGTTCGCCACAA Gamma
GAGATTTCCGCCT Delta
GAGATCTCCGCCC Epsilon


In [2]:
from Bio.Phylo.TreeConstruction import DistanceCalculator

In [3]:
DistanceCalculator.dna_models

['blastn', 'trans']

In [4]:
# calculate the distances
calculator = DistanceCalculator('identity')
distance = calculator.get_distance(alignments)
print(distance)

Alpha	0
Beta	0.23076923076923073	0
Gamma	0.3846153846153846	0.23076923076923073	0
Delta	0.5384615384615384	0.5384615384615384	0.5384615384615384	0
Epsilon	0.6153846153846154	0.3846153846153846	0.46153846153846156	0.15384615384615385	0
	Alpha	Beta	Gamma	Delta	Epsilon


In [5]:
s1 = str(alignments[0].seq)
s2 = str(alignments[1].seq)
print(s1)
print(s2)

AACGTGGCCACAT
AAGGTCGCCACAC


## Identity matrix (Hamming distance)

In [6]:
def hamming_distance(s1, s2):
    sum = 0
    for el1, el2 in zip(s1, s2):
        sum += (el1 != el2)
        
    score = sum / len(s1)
    return score

In [7]:
hamming_distance(s1, s2)

0.23076923076923078

## Blast matrix

In [8]:
calculator = DistanceCalculator('blastn')
distance = calculator.get_distance(alignments)
print(distance)

Alpha	0
Beta	0.41538461538461535	0
Gamma	0.6923076923076923	0.41538461538461535	0
Delta	0.9692307692307692	0.9692307692307692	0.9692307692307692	0
Epsilon	1.1076923076923078	0.6923076923076923	0.8307692307692307	0.27692307692307694	0
	Alpha	Beta	Gamma	Delta	Epsilon


In [9]:
calculator.dna_alphabet

['A', 'T', 'C', 'G']

In [10]:
print(calculator.dna_matrices["blastn"])

[[5], [-4, 5], [-4, -4, 5], [-4, -4, -4, 5]]


In [11]:
calculator.skip_letters

('-', '*')

+5 for a match, -4 for a mismatch, 0 for a gap

In [12]:
def get_distance(seq1, seq2, matrix, skip_letters):
    # Returns a value between 0 (identical sequences) and 1 (completely 
    #    different, or seq1 is an empty string.)"""
    
    score = 0
    max_score = 0
    for i in range(0, len(seq1)):
        l1 = seq1[i] 
        l2 = seq2[i] 
        
        if l1 in skip_letters or l2 in skip_letters: 
            continue
        
        score += matrix[l1, l2]
        max_score += matrix[l1, l1]
    
    print(score)
    print(max_score)
    return 1 - (score / max_score)    

In [13]:
print(s1)
print(s2)

AACGTGGCCACAT
AAGGTCGCCACAC


In [14]:
from Bio.Phylo.TreeConstruction import _Matrix

matrix = _Matrix(calculator.dna_alphabet, calculator.dna_matrices['blastn']) 
get_distance(s1, s2, matrix, calculator.skip_letters)

38
65


0.41538461538461535

## Transition Transversion Matrix

Paraphrased from https://en.wikipedia.org/wiki/Substitution_matrix

"Transition" is used to indicate substitutions that are between the two-ring purines (A → G and G → A) or are between the one-ring pyrimidines (C → T and T → C)<br>

"Transversion" is the term used to indicate the slower-rate substitutions that change a purine to a pyrimidine or vice versa (A ↔ C, A ↔ T, G ↔ C, and G ↔ T)

In [15]:
calculator = DistanceCalculator('trans')
distance = calculator.get_distance(alignments)
print(distance)

Alpha	0
Beta	0.3717948717948718	0
Gamma	0.7051282051282051	0.42307692307692313	0
Delta	0.8333333333333334	0.7307692307692308	0.8846153846153846	0
Epsilon	0.9230769230769231	0.5512820512820513	0.7948717948717949	0.17948717948717952	0
	Alpha	Beta	Gamma	Delta	Epsilon


In [16]:
print(calculator.dna_matrices["trans"])

[[6], [-5, 6], [-5, -1, 6], [-1, -5, -5, 6]]


In [17]:
calculator.skip_letters

('-', '*')

+6 for a match, -5 for a transversion, -1 for a transition, 0 for a gap

In [18]:
print(s1)
print(s2)

AACGTGGCCACAT
AAGGTCGCCACAC


In [19]:
matrix = _Matrix(calculator.dna_alphabet, calculator.dna_matrices['trans']) 
get_distance(s1, s2, matrix, calculator.skip_letters)

49
78


0.3717948717948718